# Working with Relational Databases and SQL

**1**. 75 points

Convert the flat file data in `data/flat.csv` into a well-structured relational database in SQLite3 stored as `data/faculty.db`. Note - salary information is confidential and should be kept in a separate table from other personal data.

In [1]:
import sqlite3, pandas as pd, numpy as np

In [2]:
flat = pd.read_csv("data/flat.csv")

In [3]:
con = sqlite3.connect("data/faculty.db")
cur = con.cursor()

Numpy is using some custom integer data types to efficiently pack data into memory. Since these types aren't familiar to sqlite, you'll have to tell it about how to handle them, beforehand:

In [4]:
for t in (np.int8, np.int16, np.int32, np.int64, np.uint8, np.uint16, np.uint32, np.uint64):
    sqlite3.register_adapter(t, float)

### Create tables and fill in from flat.csv

In [5]:
def populate_table(df):
    # Takes a dataframe and returns a list of tuples containing each row
    return(list(map(tuple, df.values)))

Person Table:

In [6]:
cur.execute("DROP TABLE IF EXISTS person")
cur.execute("""CREATE TABLE person (
    
    person_id INTEGER PRIMARY KEY, 
    first_name varchar(255), 
    last_name varchar(255),
    age INTEGER,
    height INTEGER,
    weight INTEGER,
    country_code varchar(2),
    FOREIGN KEY (country_code) REFERENCES nation (country_code),
    FOREIGN KEY (person_id) REFERENCES name (person_id)
    FOREIGN KEY (person_id) REFERENCES salary (salary_id),
    FOREIGN KEY (person_id) REFERENCES person_language (person_id)
    );""")

In [7]:
cur.executemany("INSERT INTO person (first_name, last_name, age, height, weight, country_code) VALUES (?, ?, ?, ?, ?, ?);", 
                populate_table(flat[["first", "last", "age", "height", "weight", "code"]]))

Salary Table:

In [8]:
cur.execute("DROP TABLE IF EXISTS salary")
cur.execute("""CREATE TABLE salary (

    salary_id INTEGER PRIMARY KEY,
    salary_amount INTEGER,
    FOREIGN KEY (salary_id) REFERENCES person (person_id)
    )""")

In [9]:
# Note: Make sure you've run the code at the beginning that instructs sqlite how to handle numpy integers
# Otherwise, this code will populate the table with byte values of the numpy integers
cur.executemany("INSERT INTO salary (salary_amount) VALUES (?);", populate_table(flat[["salary"]]))

Nation Table:

In [10]:
cur.execute("DROP TABLE IF EXISTS nation")
cur.execute("""CREATE TABLE nation (

    country_code varchar(255) PRIMARY KEY,
    country_name varchar(255),
    nationality varchar(255)
    )""")

Notice the errors:

In [11]:
tmp = flat[["code", "country", "nationality"]].drop_duplicates().copy()
tmp[4:6]

,code,country,nationality
5,CM,Cambodia,Cambodian
6,CM,Cameroon,Cameroonian


In [12]:
tmp[8:9]

,code,country,nationality
11,CH,China,Chinese


In [13]:
tmp[16:17]

,code,country,nationality
25,CH,Switzerland,Swiss


Notice that there are two CMs, one for Cambodia and one for Cameroon. Cliburn mentioned that this is an error, therefore I will change Cambodia to KH.
Additionally, there are two CH's: one for China and one for Switzerland. I will therefore change China to CN.

In [14]:
tmp.iloc[4, 0] = "KH"
tmp.iloc[8, 0] = "CN"

In [15]:
cur.executemany("INSERT INTO nation (country_code, country_name, nationality) VALUES (?, ?, ?);", 
                populate_table(tmp)) 

Person_Language Table:

In [16]:
cur.execute("DROP TABLE IF EXISTS person_language")
cur.execute("""CREATE TABLE person_language (
    
    person_id     INTEGER,
    language_id INTEGER,
    FOREIGN KEY (person_id) REFERENCES person (person_id),
    FOREIGN KEY (language_id) REFERENCES language (language_id)
    )""")

In [17]:
# Create an index column for the proper mapping of persons
flat['pid'] = range(1, len(flat) + 1)

In [18]:
# Create a dataframe with unique languages
languages = flat.melt(id_vars=["name"], value_vars=["language1", "language2", "language3"])[["value"]].dropna().drop_duplicates()
# Create an index column for the proper mapping of languages
languages['lid'] = range(1, len(languages) + 1)

In [19]:
person_languages = flat.melt(id_vars=["pid", "name"], value_vars=["language1", "language2", "language3"]).dropna()

In [20]:
# Our joint table for person_language table
pid_lid = person_languages.merge(languages, on = "value", how = "inner").sort_values("pid")[['pid', 'lid']]

In [21]:
cur.executemany("INSERT INTO person_language (person_id, language_id) VALUES (?, ?);",
               populate_table(pid_lid))

Language Table:

In [22]:
cur.execute("DROP TABLE IF EXISTS language")
cur.execute("""CREATE TABLE language (
    language_id INTEGER PRIMARY KEY,
    language_name varchar(255)
    )""")
# FOREIGN KEY (language_code) REFERENCES person_language (language_code)

In [23]:
cur.executemany("INSERT INTO language (language_name) VALUES (?);", 
               populate_table(languages[["value"]]))

### Check if tables function properly

Check if person-person_language-language relationship (many to many) works:

In [24]:
cur.execute("""SELECT first_name, last_name, language_name 
            FROM person 
            INNER JOIN person_language
                ON person.person_id = person_language.person_id
            INNER JOIN language
                ON person_language.language_id = language.language_id
            LIMIT 10;""")
cur.fetchall()

[('Aaron', 'Alexander', 'Haskell'),
 ('Aaron', 'Kirby', 'GNU Octave'),
 ('Aaron', 'Kirby', 'Falcon'),
 ('Aaron', 'Kirby', 'haXe'),
 ('Abram', 'Allen', 'TypeScript'),
 ('Abram', 'Boyer', 'Io'),
 ('Abram', 'Boyer', 'Lua'),
 ('Abram', 'Boyer', 'Falcon'),
 ('Adaline', 'Barry', 'Racket'),
 ('Adam', 'Lawrence', 'Perl')]

Check if person-salary relationship (one to one) works:

In [25]:
cur.execute("""SELECT first_name, last_name, salary_amount 
            FROM person 
            INNER JOIN salary
            ON person.person_id = salary.salary_id
            LIMIT 10;""")
cur.fetchall()

[('Aaron', 'Alexander', 151000),
 ('Aaron', 'Kirby', 80000),
 ('Abram', 'Allen', 75000),
 ('Abram', 'Boyer', 76000),
 ('Adaline', 'Barry', 122000),
 ('Adam', 'Lawrence', 114000),
 ('Adam', 'Rush', 103000),
 ('Adan', 'Brown', 63000),
 ('Adelle', 'Duffy', 67000),
 ('Adena', 'Holland', 117000)]

Check if person-nation relationship (one to many) works:

In [26]:
cur.execute("""SELECT first_name, last_name, nationality 
            FROM person 
            INNER JOIN nation
            ON person.country_code = nation.country_code
            LIMIT 10;""")
cur.fetchall()

[('Aaron', 'Alexander', 'British'),
 ('Aaron', 'Kirby', 'Spanish'),
 ('Abram', 'Allen', 'Italian'),
 ('Abram', 'Boyer', 'Italian'),
 ('Adaline', 'Barry', 'Uruguayan'),
 ('Adam', 'Lawrence', 'Cameroonian'),
 ('Adam', 'Rush', 'Cameroonian'),
 ('Adan', 'Brown', 'Belgian'),
 ('Adelle', 'Duffy', 'French'),
 ('Adena', 'Holland', 'Italian')]

In [27]:
con.commit()

**2**. 25 points

We want to find potential mentors for Abram	Boyer. Find all faculty members who know one or more of the same languages as Abram Boyer whose salary is at least $50,000 higher than his using SQL statements. Assume that the only information you have is that you need to find mentors meeting the criteria for the faculty member named `Abram Boyer`. In other words, the ONLY hard coded terms in your SQL query are `Abram` and `Boyer` and the salary differential.

You can use the `sql` magic extension or the `sqlite3` driver for this question.

How much does Abram Boyer make?

In [28]:
cur.execute("""SELECT first_name, last_name, salary_amount
            FROM person
            INNER JOIN salary
                ON person.person_id = salary.salary_id
            WHERE person.first_name = 'Abram'
            AND person.last_name = 'Boyer'""")
cur.fetchall()

[('Abram', 'Boyer', 76000)]

What languages does Abram Boyer know?

In [29]:
cur.execute("""SELECT language_name
                FROM language
                INNER JOIN person_language
                    ON language.language_id = person_language.language_id
                INNER JOIN person
                    ON person_language.person_id = person.person_id
                WHERE person.first_name = 'Abram'
                AND person.last_name = 'Boyer'""")
cur.fetchall()

[('Io',), ('Lua',), ('Falcon',)]

In [30]:
cur.execute("""SELECT DISTINCT first_name, last_name, salary_amount
            FROM person
            INNER JOIN salary
                ON person.person_id = salary.salary_id
            INNER JOIN person_language
                ON person.person_id = person_language.person_id
            INNER JOIN language
                ON person_language.language_id = language.language_id
            WHERE salary.salary_amount >= 50000 + 
                (SELECT salary_amount
                FROM salary
                INNER JOIN person
                    ON person.person_id = salary.salary_id
                WHERE person.first_name = 'Abram'
                AND person.last_name = 'Boyer')
            AND language.language_name IN 
                (SELECT language_name
                FROM language
                INNER JOIN person_language
                    ON language.language_id = person_language.language_id
                INNER JOIN person
                    ON person_language.person_id = person.person_id
                WHERE person.first_name = 'Abram'
                AND person.last_name = 'Boyer')
            """)
cur.fetchall()

[('Dawne', 'Knapp', 185000),
 ('Dong', 'Walton', 154000),
 ('Ellsworth', 'Jackson', 155000),
 ('Everette', 'Garrison', 126000),
 ('Guillermo', 'Herring', 162000),
 ('Hai', 'Myers', 130000),
 ('Jama', 'Ware', 175000),
 ('Janessa', 'Chan', 170000),
 ('Jarrod', 'Hall', 143000),
 ('Jesenia', 'Wynn', 145000),
 ('Kermit', 'Pruitt', 139000),
 ('Kip', 'Glenn', 127000),
 ('Lenard', 'Holt', 129000),
 ('Leonia', 'Johnson', 165000),
 ('Lurlene', 'Martin', 142000),
 ('Natashia', 'Dodson', 140000),
 ('Retta', 'Stewart', 143000),
 ('Reuben', 'Gay', 179000),
 ('Sang', 'Battle', 134000),
 ('Seth', 'Simpson', 163000),
 ('Timmy', 'Browning', 155000),
 ('Tobias', 'Burnett', 152000),
 ('Ula', 'Mayo', 129000),
 ('Vanita', 'Porter', 130000),
 ('Zane', 'Calhoun', 127000)]

In [31]:
con.close()